Proyecto creado por Sergio Sánchez Zahiños (sersanch)

Importamos los módulos. Usaremos el modelo "en_core_web_sm" de Spacy

In [ ]:
import pandas as pd
import spacy
import re
from collections import Counter

nlp = spacy.load("en_core_web_sm")

Importamos CSV y lo convertimos en dataframe (pandas)

In [ ]:
df = pd.read_csv(r'comments_user.csv')[['v1', 'v2']]

Creamos un dataframe nuevo con los mensajes ya clasificados como spam

In [48]:
df_spam = df.loc[df['v1'] == 'spam']

Creamos una variables que contiene la concatenación de cada mensaje de spam, quitando números

In [71]:
spam_text = ''
for index, row in df_spam.iterrows():
  spam_text += row['v2'] + " "
spam_text = re.sub(r'\d', '', spam_text)

Tokenizamos las palabras quitando los carácteres que no nos interesan, como por ejemplo las "stop words"

In [73]:
nlp_text = nlp(spam_text)
filtered_words = [token.text for token in nlp_text if not token.is_stop and not token.is_punct]

Hacemos una lista con las palabras (únicas) en los mensajes de spam

In [ ]:
word_freq = Counter(filtered_words)
common_words = word_freq.most_common()
common_words = list(filter(lambda x: len(x[0]) >= 3, common_words))
common_words = list(set([x[0].upper() for x in common_words]))

Creamos una función que clasifique un texto en si es SCAM o no. Si contiene al menos un 25% de palabras encontradas en los mensajes de spam, se clasifica como spam.

In [93]:
def is_scam(text: str, forbidden_words):
  count = 0
  words_number = len(text.split())
  text = text.upper()
  for word in forbidden_words:
    if word in text:
      count += 1
  if count >= words_number * 0.25:
    return True
  return False

Creamos la tabla final en la que guardaremos los resultados. Tendrá una columna 'comment' con el comentario y otra 'type', indicando si es scam o safe.

In [94]:
final_table = pd.DataFrame(columns=['comment', 'type'])
for index, row in df.iterrows():
  if is_scam(row['v1'], common_words) or row['v1'] == 'spam':
    comment_type = 'scam'
  else:
    comment_type = 'safe'
  final_table.loc[index] = [row['v2'], comment_type]

Transformamos la tabla a formato CSV e imprimimos

In [ ]:
final_csv = final_table.to_csv()
print(final_csv)